# Imports

In [ ]:
!pip install statannot

In [ ]:
import os.path

import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import statannot
from statannotations.Annotator import Annotator

In [ ]:
from restore_data import recover_data

In [ ]:
import logging

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s :: %(levelname)s :: %(message)s')
log = logging.getLogger("DataImport")
log.setLevel(logging.DEBUG)

In [ ]:
from IPython.display import set_matplotlib_formats
%matplotlib inline

In [ ]:
plt.style.context('seaborn-paper')
# plt.style.use('seaborn-notebook')
# plt.style.use('seaborn-poster')
# plt.style.use('seaborn-talk')
set_matplotlib_formats('svg')
plt.rcParams.update(plt.rcParamsDefault)


linewidth = 0.25
titlesize = 'medium'
labelsize = 'small'
ticksize = 'x-small'

markersize = 10
scattersize = 5

palette_stages='gist_heat'

plt.rcParams.update({
    'figure.figsize': [5.9/2, 5.9/2 * 2/3],

    'text.usetex': False,
    'font.size': 10,
    'font.family': 'sans-serif',
    'font.sans-serif': 'Helvetica',

    'figure.titlesize': titlesize,
    'legend.title_fontsize': labelsize,
    'legend.fontsize': ticksize,
    'axes.labelsize': labelsize,
    'xtick.labelsize': ticksize,
    'ytick.labelsize': ticksize,
    'ytick.labelsize': ticksize,

    'figure.autolayout': False,

    'axes.linewidth': linewidth,
    'xtick.major.width': linewidth,
    'xtick.minor.width': 0.8*linewidth,
    'ytick.major.width': linewidth,
    'ytick.minor.width': 0.8*linewidth,
    'grid.linewidth': linewidth,

    'patch.linewidth': linewidth,

    'lines.markersize': scattersize
})

In [ ]:
basepath = os.path.dirname('')
basepath = "/home/julian/ownCloud/Institution/Collaboration_NCBS_UNIGE"
data_basedir = "data"
rel_paths=[
    'Live_imaging/Control/E10_Control_1_2',
    'Live_imaging/Control/E10_Control_2_1',
    'Live_imaging/Control/E10_Control_3',
    'Live_imaging/Control/E10_Control_99',
    'Live_imaging/ML7/E10_ML7_1',
    'Live_imaging/ML7/E10_ML7_2_1',
    'Live_imaging/PTX/E10_PTX_1_1',
    'Live_imaging/PTX/E10_PTX_2_1',
]
data_path = "102_treated_data"
load_file_prefix='summary'
save_file_prefix='summary_live_imaging'

min_tracking_period = 5 # number of frames

In [ ]:
figure_path = 'analysis/1_figures'
figure_path = os.path.abspath(os.path.join(basepath, figure_path))

In [ ]:
paths = []
for p in rel_paths:
    paths.append(os.path.join(basepath, data_basedir, p, data_path))
print(paths)
cells, bonds = recover_data(paths=paths, save_file_prefix=load_file_prefix)
cells = cells.loc[~cells['is_border_cell']]

HC = cells.loc[cells['is_HC']]
SC = cells.loc[~cells['is_HC']]

# Analysis

In [ ]:
cilia = pd.DataFrame()

cilia['global_id'] = cells['global_id_cells']
cilia['filename'] = cells['filename']
cilia['file_id'] = cells['file_id']
cilia['filepath'] = cells['filepath']
cilia['X'] = cells['center_x_cells'] + cells['cilium_DX']
cilia['Y'] = cells['center_y_cells'] + cells['cilium_DY']

In [ ]:
bonds['center_x_bonds'] = (bonds['vx_1_x'] + bonds['vx_2_x']) / 2
bonds['center_y_bonds'] = (bonds['vx_1_y'] + bonds['vx_2_y']) / 2

In [ ]:
def closest_cilium(bond, *, cilia):
    """Assign to a cell the cilium that is the closest in space

    Return:
        - global id of cilium
        - distance of cilium to cell center in units of cell radius
        - X position of cilium
        - Y position of cilium
        - Number of cilia that have distance rho < 1 to cell center

    """
    _cilia = cilia.loc[cilia['file_id'] == bond['file_id']].reset_index()
    if _cilia.empty or bond['type'] == 'border_plus_one':
        cilium = pd.Series(data=None, index=_cilia.columns, dtype=object)
        cilium['rho'] = np.nan
        return cilium

    r = (  (_cilia['X'] - bond['center_x_bonds'])**2
         + (_cilia['Y'] - bond['center_y_bonds'])**2)**0.5

    id = r.loc[r == r.min()].index[0]
    cilium = _cilia.iloc[id]
    cilium['rho'] = r.iloc[id]

    return cilium

Experiment = 'Control'
sample_id = 2
frame = cells.loc[cells['Experiment'] == Experiment]
frame = frame.loc[frame['sample_id'] == sample_id]
frame_bonds = bonds.loc[bonds['Experiment'] == Experiment]
frame_bonds = frame_bonds.loc[frame_bonds['sample_id'] == sample_id]

__cilia = cilia.loc[~np.isnan(cilia['X'])]
assigned_cilia = bonds.apply(closest_cilium, axis=1, cilia=__cilia)


In [ ]:
bonds['cilium_X'] = assigned_cilia['X']
bonds['cilium_Y'] = assigned_cilia['Y']
bonds['cilium_DX'] = bonds['center_x_bonds'] - bonds['cilium_X']
bonds['cilium_DY'] = bonds['center_y_bonds'] - bonds['cilium_Y']
bonds['cilium_rho'] = assigned_cilia['rho']

In [ ]:
def split_coords_vx(coords):
    x = []
    if type(coords) is np.ndarray:
        return coords
    if type(coords) is str:
        for _x in coords[1:-1].split(' '):
            if _x == '':
                continue
            x.append(_x)

        return np.array(x, dtype=int)
    if type(coords) is float and np.isnan(coords):
        return np.array([], dtype=int)
    raise RuntimeError("Cannot split vertex coords of cell of type {} ({})"
        "".format(type(coords), coords))

In [ ]:
bonds['cilium_angular_distance'] = np.nan

cells['vx_coords_x_cells'] = cells['vx_coords_x_cells'].apply(split_coords_vx)
cells['vx_coords_y_cells'] = cells['vx_coords_y_cells'].apply(split_coords_vx)

for id, cell in cells.loc[~np.isnan(cells['cilium_X'])].iterrows():
    if not cell['is_HC']:
        continue

    cx = cell['center_x_cells']
    cy = cell['center_y_cells']

    rho = (cell['area_cells']/ math.pi)**0.5
    px = cell['cilium_rho_corrected'] * np.cos(cell['cilium_phi_corrected'])
    py = cell['cilium_rho_corrected'] * np.sin(cell['cilium_phi_corrected'])

    # HS bonds
    for bond_id in cell['global_id_bonds']:
        bond = bonds.loc[bonds['global_id_bonds']==bond_id].iloc[0]

        bx = cx - bond['center_x_bonds']
        by = cy - bond['center_y_bonds']

        angle = np.arccos((px * bx + py * by) / ((bx**2+by**2)**0.5 * (px**2+py**2)**0.5))

        bonds.loc[bonds['global_id_bonds']==bond_id, 'cilium_angular_distance'] = angle

    # SS bonds
    cnt = 0
    for vx, vy in zip(cell['vx_coords_x_cells'], cell['vx_coords_y_cells']):
        __bonds = bonds.loc[
            (bonds['filename']==cell['filename']) & 
            (bonds['type']=='SS')
        ]
        if __bonds.empty:
            raise
        b = __bonds.loc[
            ((__bonds['vx_1_x']==vx) | (__bonds['vx_2_x']==vx)) &
            ((__bonds['vx_1_y']==vy) | (__bonds['vx_2_y']==vy))
        ]
        if b.empty:
            continue
        if len(b) > 1:
            print("More than one SS bond found: {}".format(b))
            continue
        b = b.iloc[0]
        cnt += 1

        bx = cx - vx
        by = cy - vy

        angle = np.arccos((px * bx + py * by) / ((bx**2+by**2)**0.5 * (px**2+py**2)**0.5))
        
        if not np.isnan(b['cilium_angular_distance']):
            angle = max(angle, b['cilium_angular_distance'])

        bonds.loc[bonds['global_id_bonds']==b['global_id_bonds'], 'cilium_angular_distance'] = angle
    if cnt == 0:
        print(cell[['filename', 'global_id_cells']])

print(bonds.groupby(by='filename').apply(lambda bonds: bonds['cilium_angular_distance'].count()).to_string())

In [ ]:
_bonds = bonds.loc[bonds['type'] != 'border_plus_one']
_bonds = _bonds.loc[_bonds['type'] != 'border']
_bonds = _bonds.loc[_bonds['type'] != 'HH']

tracked_bonds = pd.DataFrame(
    _bonds.groupby(by='global_track_id_bonds').apply(
        lambda tracked_bond: tracked_bond['type'].unique()[0]),
    columns=['type']
)


tracked_bonds['tracked_frames'] = bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['time'].count()
)
tracked_bonds['tracked_time'] = _bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['time'].max() - tracked_bond['time'].min()
)


tracked_bonds['Experiment'] = _bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['Experiment'].unique()[0]
)
tracked_bonds['position'] = _bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['position'].unique()[0]
)
tracked_bonds['stage'] = _bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['stage'].unique()[0]
)
tracked_bonds['sample_id'] = _bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['sample_id'].unique()[0]
)
tracked_bonds['filename'] = _bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['filename'].unique()[0].split('00')[0][:-5]
)
tracked_bonds['filepath'] = _bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['filepath'].unique()[0]
)


tracked_bonds['bond_length_mean'] = _bonds.groupby(by='global_track_id_bonds').apply(
    lambda tracked_bond: tracked_bond['bond_length_in_px'].mean()
)

tracked_bonds['bond_length_stddev_norm'] = _bonds.groupby(by='global_track_id_bonds')['bond_length_in_px'].std() / tracked_bonds['bond_length_mean']

tracked_bonds['bond_length_variance_norm'] = _bonds.groupby(by='global_track_id_bonds')['bond_length_in_px'].var() / tracked_bonds['bond_length_mean']

def running_stddev_norm(tracked_bond):
    tb_length = tracked_bond['bond_length_in_px'].rolling(10)
    return (tb_length.std() / tb_length.mean()).mean()

tracked_bonds['bond_length_running_stddev_norm'] = _bonds.groupby(by='global_track_id_bonds').apply(
    running_stddev_norm
)


tracked_bonds['bond_orientation_mean'] = _bonds.groupby(by='global_track_id_bonds')['bond_orientation'].mean()

tracked_bonds['bond_orientation_stddev'] = _bonds.groupby(by='global_track_id_bonds')['bond_orientation'].std()

def classify_orientation__by_3(tracked_bond):
    orientation = tracked_bond['bond_orientation_mean']

    if orientation < math.pi / 6:
        return 'horizontal'
    elif orientation < 2 * math.pi / 6:
        return 'diagonal'
    elif orientation < 4 * math.pi / 6:
        return 'vertical'
    elif orientation < 5 * math.pi / 6:
        return 'diagonal'
    else:
        return 'horizontal'
        
def classify_orientation__by_2(tracked_bond):
    orientation = tracked_bond['bond_orientation_mean']

    if orientation < math.pi / 2:
        return 'horizontal'
    # elif orientation < 3 * math.pi / 4:
    else:
        return 'vertical'
    # else:
    #     return 'horizontal'


tracked_bonds['bond_orientation_class'] = tracked_bonds.apply(
    classify_orientation__by_3,
    axis=1
)
tracked_bonds['bond_orientation_class__by_2'] = tracked_bonds.apply(
    classify_orientation__by_2,
    axis=1
)

# Classify bonds
tracked_bonds.loc[tracked_bonds['type'] == 'HS', 'class__'] = 'HS'
tracked_bonds.loc[tracked_bonds['type'] == 'SS', 'class__'] = tracked_bonds['bond_orientation_class']

tracked_bonds.loc[tracked_bonds['class__'] == 'horizontal', 'class__'] = 'SS_horizontal'
tracked_bonds.loc[tracked_bonds['class__'] == 'diagonal', 'class__'] = 'SS_other'
tracked_bonds.loc[tracked_bonds['class__'] == 'vertical', 'class__'] = 'SS_other'

tracked_bonds['class'] = tracked_bonds['class__']
tracked_bonds.loc[tracked_bonds['class'] == 'SS_horizontal', 'class'] = 'SS || P-D'
tracked_bonds.loc[tracked_bonds['class'] == 'SS_other', 'class'] = 'SS non P-D'


# polarity
tracked_bonds['cilium_rho__mean'] = _bonds.groupby(by='global_track_id_bonds')['cilium_rho'].mean()
tracked_bonds['cilium_rho__max'] = _bonds.groupby(by='global_track_id_bonds')['cilium_rho'].max()
tracked_bonds['cilium_rho__min'] = _bonds.groupby(by='global_track_id_bonds')['cilium_rho'].min()
tracked_bonds['cilium_rho__std'] = _bonds.groupby(by='global_track_id_bonds')['cilium_rho'].std()
tracked_bonds['cilium_angular_distance__mean'] = _bonds.groupby(by='global_track_id_bonds')['cilium_angular_distance'].mean()
tracked_bonds['cilium_angular_distance__max'] = _bonds.groupby(by='global_track_id_bonds')['cilium_angular_distance'].max()
tracked_bonds['cilium_angular_distance__min'] = _bonds.groupby(by='global_track_id_bonds')['cilium_angular_distance'].min()
tracked_bonds['cilium_angular_distance__std'] = _bonds.groupby(by='global_track_id_bonds')['cilium_angular_distance'].std()


# Exclude short trackings
tracked_bonds = tracked_bonds.loc[tracked_bonds['tracked_frames'] >= 2]
_tracked_bonds = tracked_bonds

tracked_bonds = tracked_bonds.loc[tracked_bonds['tracked_frames'] >= min_tracking_period]
print(tracked_bonds.groupby(by=['type', 'Experiment']).apply(lambda bs: (bs.shape[0], bs['tracked_frames'].mean())))

print('SS bonds:')
print(tracked_bonds.loc[tracked_bonds['type'] == 'SS'].groupby(by=['bond_orientation_class', 'Experiment']).apply(lambda bs: (bs.shape[0], bs['tracked_frames'].mean())))


In [ ]:
tracked_cells = pd.DataFrame(
    cells.groupby(by='global_track_id_cells').apply(
        lambda tracked_cell: tracked_cell['is_HC'].unique()[0]),
    columns=['is_HC']
)

tracked_cells['tracked_frames'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_cell: tracked_cell['time'].count()
)

tracked_cells['tracked_time'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_cell: tracked_cell['time'].max() - tracked_cell['time'].min()
)


tracked_cells['Experiment'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_cell: tracked_cell['Experiment'].unique()[0]
)
tracked_cells['position'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_cell: tracked_cell['position'].unique()[0]
)
tracked_cells['stage'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_cell: tracked_cell['stage'].unique()[0]
)
tracked_cells['sample_id'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_cell: tracked_cell['sample_id'].unique()[0]
)
tracked_cells['filename'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_cell: tracked_cell['filename'].unique()[0].split('.')[0][:-5]
)
tracked_cells['filepath'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_cell: tracked_cell['filepath'].unique()[0]
)



tracked_cells['normalized_area_mean'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_bond: tracked_bond['normalized_area_cells'].mean()
)
tracked_cells['normalized_area_stddev'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_bond: tracked_bond['normalized_area_cells'].std()
)

tracked_cells['perimeter_length_mean'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_bond: tracked_bond['perimeter_length'].mean()
)
tracked_cells['perimeter_length_stddev'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_bond: tracked_bond['perimeter_length'].std()
)

tracked_cells['shape_index_mean'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_bond: tracked_bond['shape_index'].mean()
)
tracked_cells['shape_index_stddev'] = cells.groupby(by='global_track_id_cells').apply(
    lambda tracked_bond: tracked_bond['shape_index'].std()
)


# Exclude short trackings
tracked_cells = tracked_cells.loc[tracked_cells['tracked_frames'] >= 2]
_tracked_cells = tracked_cells

tracked_cells = tracked_cells.loc[tracked_cells['tracked_frames'] >= min_tracking_period]

In [ ]:
cells.to_csv("{}/{}/{}_cells.csv".format(basepath, data_basedir, save_file_prefix))
bonds.to_csv("{}/{}/{}_bonds.csv".format(basepath, data_basedir, save_file_prefix))
tracked_bonds.to_csv("{}/{}/{}_tracked_bonds.csv".format(basepath, data_basedir, save_file_prefix))
tracked_cells.to_csv("{}/{}/{}_tracked_cells.csv".format(basepath, data_basedir, save_file_prefix))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(5, 8.5))

sns.violinplot(
    # data=tracked_bonds.loc[tracked_bonds['bond_length_stddev'] < 0.7],
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='type',
    hue_order=['HS', 'SS'],
    inner='box',
    bw=0.1
)

statannot.add_stat_annotation(
    ax,
    # data=tracked_bonds.loc[tracked_bonds['bond_length_stddev'] < 0.7],
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='type',
    hue_order=['HS', 'SS'],
    box_pairs=[
        (("Control", "SS"), ("Control", "HS")),
        (("ML7", "SS"), ("ML7", "HS")),
        (("PTX", "SS"), ("PTX", "HS")),
        (("Control", "SS"), ("ML7", "SS")),
        (("Control", "HS"), ("ML7", "HS")),
        (("Control", "HS"), ("PTX", "HS")),
    ],
    test="t-test_welch",
    text_format="simple",
    loc="outside",
)

ax.set_ylabel('rel. length fluctuation')
ax.set_ylim([0, 1.])
ax.legend(loc='upper right')


plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation.svg"))

In [ ]:
print(tracked_bonds['type'].unique())

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=[1.6, 2.0])
plt.tight_layout()

sns.violinplot(
    data=tracked_bonds.loc[tracked_bonds['Experiment'] != 'PTX'],
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='type',
    hue_order=['HS', 'SS'],
    inner='box',
    palette=['red', 'gray'],
    linewidth=0.25,

    # bw=0.15
)

box_pairs=[
    (("Control", "HS"), ("Control", "SS")),
    (("ML7", "SS"), ("ML7", "HS")),
    (("Control", "SS"), ("ML7", "SS")),
    (("Control", "HS"), ("ML7", "HS")),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=tracked_bonds.loc[tracked_bonds['Experiment'] != 'PTX'],
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='type',
    hue_order=['HS', 'SS'],
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.25,
    fontsize=labelsize,
)
annotator.apply_and_annotate()


ax.set_ylabel('rel. length fluctuation')
ax.set_xlabel('')
ax.set_ylim([0, 1.25])
ax.set_yticks([0., 0.5, 1.])

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['HC-SC', 'SC-SC'], title='', loc='upper right')

plt.savefig("{}/{}".format(figure_path,
                           "bond_length_fluctuation_Control_ML7.svg"))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(1.4, 1.4))
plt.tight_layout()

data = tracked_bonds.loc[tracked_bonds['type'] == 'SS']
data = data.loc[data['Experiment'] == 'Control']
limit = 20 / 180 * math.pi

data['bond_orientation_class__custom'] = np.where(
    data['bond_orientation_mean'] < limit,
    'aligned',
    np.where(
        data['bond_orientation_mean'] > math.pi - limit,
        'aligned',
        'other'
    )
)
print(data.columns)
print(np.unique(data['filename']))
print(data.groupby(by='bond_orientation_class__custom').count()['type'])

sns.violinplot(
    data=data,
    x='bond_orientation_class__custom',
    order=['aligned', 'other'],
    y='bond_length_stddev_norm',
    hue='type',
    inner='box',
    palette=['gray', 'gray'],
    linewidth=linewidth,
    # bw=0.15
)

annotator = Annotator(
    ax,
    [
        ('aligned', 'other'),
    ],
    data=data,
    x='bond_orientation_class__custom',
    order=['aligned', 'other'],
    y='bond_length_stddev_norm',
)
annotator.configure(
    test="t-test_welch",
    # test='Mann-Whitney',
    line_width=linewidth,
)
annotator.apply_and_annotate()

ax.set_xlabel('Angle to tissue axis')
ax.set_xticklabels([r'$< 30 \degree$',r'$\geq 30 \degree$'])

ax.set_ylabel('rel. length fluctuation')
ax.set_ylim([0, 1.])
ax.legend(loc='upper left')

plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation__Control.svg"))

fix, ax = plt.subplots(1, 1, figsize=(1.4, 1.4))
sns.histplot(
    data=data,
    x='bond_orientation_mean',
    hue='bond_orientation_class__custom',
    binrange=[0, math.pi],
    multiple='stack'
)

In [ ]:
for Experiment in tracked_bonds['Experiment'].unique():
    if Experiment == 'Control':
        continue
    fix, ax = plt.subplots(1, 1, figsize=(1.7, 1.7))
    plt.tight_layout()

    data = tracked_bonds.loc[tracked_bonds['type'] == 'SS']
    data = data.loc[(data['Experiment'] == 'Control') | (data['Experiment'] == Experiment)]
    limit = 30 / 180 * math.pi
    data['bond_orientation_class__custom'] = np.where(
        (data['bond_orientation_mean'] < limit) | (data['bond_orientation_mean'] > math.pi - limit),
        'aligned',
        'other'
    )
    print(data.columns)
    print(np.unique(data['filename']))
    print(data.groupby(by='bond_orientation_class__custom').count()['type'])

    order=['Control', Experiment]
    sns.violinplot(
        data=data,
        x='Experiment',
        order=order,
        y='bond_length_stddev_norm',
        hue='bond_orientation_class__custom',
        hue_order=['aligned', 'other'],
        inner='box',
        palette=['gray', 'whitesmoke'],
        linewidth=linewidth
        # bw=0.15
    )

    annotator = Annotator(
        ax,
        [
            (('Control', 'aligned'), ('Control', 'other')),
            ((Experiment, 'aligned'), (Experiment, 'other')),
            
            (('Control', 'aligned'), (Experiment, 'aligned')),
            (('Control', 'other'), (Experiment, 'other')),
        ],
        data=data,
        x='Experiment',
        order=order,
        y='bond_length_stddev_norm',
        hue='bond_orientation_class__custom',
        hue_order=['aligned', 'other'],
    )
    annotator.configure(
        test="t-test_welch",
        # test='Mann-Whitney',
        line_width=linewidth,
    )
    annotator.apply_and_annotate()

    # ax.set_xlabel('Angle to tissue axis')
    # ax.set_xticklabels([r'$< 30 \degree$',r'$\geq 30 \degree$'])

    ax.set_ylabel('rel. length fluctuation')
    ax.set_ylim([0, 1.])
    ax.set_yticks([0, 0.5, 1.])

    ax.set_xlabel('')
    ax.set_xticklabels(order, fontsize=labelsize)


    leg_handles = ax.get_legend_handles_labels()[0]
    ax.legend(
        leg_handles,
        [r'$< {:.0f} \degree$'.format(limit/math.pi*180), 
         r'$\geq {:.0f} \degree$'.format(limit/math.pi*180)],
        fontsize=ticksize,
        title='',
        bbox_to_anchor=(1,1.025)
    )
    # mpl.pyplot.legend()

    # ax.legend(loc='upper left')

    plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation__{}.svg".format(Experiment)))

    fix, ax = plt.subplots(1, 1, figsize=(1.4, 1.4))
    sns.histplot(
        data=data,
        x='bond_orientation_mean',
        hue='bond_orientation_class__custom',
        binrange=[0, math.pi],
        multiple='stack'
    )

In [ ]:
print()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=[6, 8])
sns.kdeplot(data=data, x='bond_length_stddev_norm', y='bond_orientation_mean', levels=15, ax=ax1)
ax1.set_xlim(0, 0.5)
ax1.set_ylim(0, math.pi)

data['angle_to_axis_mean'] = np.where(
    data['bond_orientation_mean'] < math.pi / 2,
    data['bond_orientation_mean'],
    math.pi - data['bond_orientation_mean']
)

sns.kdeplot(data=data, x='bond_length_stddev_norm', y='angle_to_axis_mean', levels=15, ax=ax2)
ax2.set_xlim(0, 0.5)
ax2.set_ylim(0, math.pi/2*1.1)

data['bond_orientation_mean__shift'] = np.where(
    data['bond_orientation_mean'] < math.pi / 2,
    data['bond_orientation_mean'],
    -(math.pi - data['bond_orientation_mean'])
)
sns.kdeplot(data=data, x='bond_length_stddev_norm', y='bond_orientation_mean__shift', levels=15, ax=ax3)
ax3.set_xlim(0, 0.5)
# ax3.set_ylim(0, math.pi/2*1.1)


In [ ]:
fig, ax = plt.subplots(1, 1, )
data = data.sort_values(by='angle_to_axis_mean')
count = data.shape[0]

for i, (idi, bond) in enumerate(data.iterrows()):
    data.loc[idi, 'orientation_class'] = np.floor(i / 60)



# data['orientation_class'] = np.floor(data['bond_orientation_mean'] / math.pi * 6)
sns.violinplot(
    data=data,
    x='orientation_class',
    # order=['aligned', 'other'],
    y='bond_length_stddev_norm',
    hue='type',
    inner='box',
    palette=['gray', 'gray']
    # bw=0.15
)
ax.set_ylim([0, 1])
annotator = Annotator(
    ax,
    [
        (0.0, 1.0),
        (1.0, 2.0),
        (2.0, 3.0),
        (3.0, 4.0),
        (4.0, 5.0),
        (5.0, 6.0),
        (6.0, 7.0),
        (7.0, 0.0),

    ],
    data=data,
    x='orientation_class',
    y='bond_length_stddev_norm',
)
annotator.configure(
    test="t-test_welch",
    # test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()

print(data.groupby(by='orientation_class').count()['type'])

fix, ax = plt.subplots(1, 1)
sns.histplot(
    data=data,
    x='bond_orientation_mean',
    hue='orientation_class',
    binrange=[0, math.pi],
    multiple='stack'
)

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(5, 8.5))

sns.violinplot(
    data=tracked_bonds.loc[tracked_bonds['type'] == 'SS'],
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='bond_orientation_class',
    hue_order=['vertical', 'diagonal', 'horizontal'],
    inner='box',
    bw=0.15
)

statannot.add_stat_annotation(
    ax,
    data=tracked_bonds.loc[tracked_bonds['type'] == 'SS'],
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='bond_orientation_class',
    hue_order=['vertical', 'diagonal', 'horizontal'],
    box_pairs=[
        (("Control", "vertical"), ("Control", "diagonal")),
        (("Control", "vertical"), ("Control", "horizontal")),

        (("ML7", "vertical"), ("ML7", "diagonal")),
        (("ML7", "vertical"), ("ML7", "horizontal")),

        (("PTX", "vertical"), ("PTX", "diagonal")),
        (("PTX", "vertical"), ("PTX", "horizontal")),

        (("Control", "vertical"), ("ML7", "vertical")),
        (("Control", "horizontal"), ("ML7", "horizontal")),

        (("Control", "vertical"), ("PTX", "vertical")),
        (("Control", "horizontal"), ("PTX", "horizontal")),
    ],
    test="t-test_welch",
    text_format="simple",
    loc="inside",
)

ax.set_ylabel('rel. length fluctuation')
ax.set_ylim([0, 1.6])
ax.legend(loc='upper right')


plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation.svg"))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(5, 8.5))

sns.violinplot(
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='class',
    hue_order=['HS', 'SS non P-D', 'SS || P-D'],
    inner='box',
    bw=0.15
)

statannot.add_stat_annotation(
    ax,
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='class__',
    hue_order=['HS', 'SS_other', 'SS_horizontal'],
    box_pairs=[
        (("Control", "HS"), ("Control", "SS_horizontal")),
        (("Control", "HS"), ("Control", "SS_other")),

        (("ML7", "HS"), ("ML7", "SS_horizontal")),
        (("ML7", "HS"), ("ML7", "SS_other")),

        (("PTX", "HS"), ("PTX", "SS_horizontal")),
        (("PTX", "HS"), ("PTX", "SS_other")),

        (("Control", "HS"), ("ML7", "HS")),
        (("Control", "SS_other"), ("ML7", "SS_other")),
        (("Control", "SS_horizontal"), ("ML7", "SS_horizontal")),

        (("Control", "HS"), ("PTX", "HS")),
        (("Control", "SS_other"), ("PTX", "SS_other")),
        (("Control", "SS_horizontal"), ("PTX", "SS_horizontal")),
    ],
    test="t-test_welch",
    text_format="simple",
    loc="inside",
)

ax.set_ylabel('rel. length fluctuation')
ax.set_ylim([0, 2])
ax.legend(loc='upper right')


plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation_new.svg"))

In [ ]:
plt.rcParams['font.size'] = 6
plt.rcParams['lines.linewidth'] = 0.75


fix, ax = plt.subplots(1, 1, figsize=(1.8, 3.5))
plt.tight_layout()

sns.violinplot(
    data=tracked_bonds.loc[tracked_bonds['Experiment'] != 'PTX'],
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='class',
    hue_order=['HS', 'SS non P-D', 'SS || P-D'],
    inner='box',
    bw=0.15
)

box_pairs=[
    (("Control", "HS"), ("Control", "SS_horizontal")),
    (("Control", "HS"), ("Control", "SS_other")),

    (("ML7", "HS"), ("ML7", "SS_horizontal")),
    (("ML7", "HS"), ("ML7", "SS_other")),

    # (("PTX", "HS"), ("PTX", "SS_horizontal")),
    # (("PTX", "HS"), ("PTX", "SS_other")),

    (("Control", "HS"), ("ML7", "HS")),
    (("Control", "SS_other"), ("ML7", "SS_other")),
    (("Control", "SS_horizontal"), ("ML7", "SS_horizontal")),

    # (("Control", "HS"), ("PTX", "HS")),
    # (("Control", "SS_other"), ("PTX", "SS_other")),
    # (("Control", "SS_horizontal"), ("PTX", "SS_horizontal")),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=tracked_bonds.loc[tracked_bonds['Experiment'] != 'PTX'],
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='class__',
    hue_order=['HS', 'SS_other', 'SS_horizontal'],
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75
)
annotator.apply_and_annotate()


ax.set_ylabel('rel. length fluctuation')
ax.set_xlabel('')
ax.set_ylim([0, 1])
ax.legend(loc='upper right')


plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation_new_no_error.svg"), bbox_inches="tight")
plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(4, 8.5))

sns.violinplot(
    data=tracked_bonds.loc[tracked_bonds['Experiment'] != 'ML7'],
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='class',
    hue_order=['HS', 'SS non P-D', 'SS || P-D'],
    inner='box',
    bw=0.15
)

box_pairs=[
    (("Control", "HS"), ("Control", "SS_horizontal")),
    (("Control", "HS"), ("Control", "SS_other")),

    # (("ML7", "HS"), ("ML7", "SS_horizontal")),
    # (("ML7", "HS"), ("ML7", "SS_other")),

    (("PTX", "HS"), ("PTX", "SS_horizontal")),
    (("PTX", "HS"), ("PTX", "SS_other")),

    # (("Control", "HS"), ("ML7", "HS")),
    # (("Control", "SS_other"), ("ML7", "SS_other")),
    # (("Control", "SS_horizontal"), ("ML7", "SS_horizontal")),

    (("Control", "HS"), ("PTX", "HS")),
    (("Control", "SS_other"), ("PTX", "SS_other")),
    (("Control", "SS_horizontal"), ("PTX", "SS_horizontal")),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=tracked_bonds.loc[tracked_bonds['Experiment'] != 'ML7'],
    x='Experiment',
    y='bond_length_stddev_norm',
    hue='class__',
    hue_order=['HS', 'SS_other', 'SS_horizontal'],
)
annotator.configure(test='Mann-Whitney',)
annotator.apply_and_annotate()

# statannot.add_stat_annotation(
#     ax,
#     data=tracked_bonds.loc[tracked_bonds['Experiment'] != 'PTX'],
#     x='Experiment',
#     y='bond_length_stddev_norm',
#     hue='class',
#     hue_order=['HS', 'SS_other', 'SS_horizontal'],
#     ,
#     test="t-test_welch",
#     # text_format="simple",
#     loc="inside",
# )

ax.set_ylabel('rel. length fluctuation')
ax.set_ylim([0, 1])
ax.legend(loc='upper right')


plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation_new_no_error__PTX.svg"))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(5, 8.5))

sns.violinplot(
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_variance_norm',
    hue='class',
    hue_order=['HS', 'SS other', 'SS horizontal'],
    inner='box',
    bw=0.15
)

statannot.add_stat_annotation(
    ax,
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_variance_norm',
    hue='class__',
    hue_order=['HS', 'SS_other', 'SS_horizontal'],
    box_pairs=[
        (("Control", "HS"), ("Control", "SS_horizontal")),
        (("Control", "HS"), ("Control", "SS_other")),

        (("ML7", "HS"), ("ML7", "SS_horizontal")),
        (("ML7", "HS"), ("ML7", "SS_other")),

        (("PTX", "HS"), ("PTX", "SS_horizontal")),
        (("PTX", "HS"), ("PTX", "SS_other")),

        (("Control", "HS"), ("ML7", "HS")),
        (("Control", "SS_other"), ("ML7", "SS_other")),
        (("Control", "SS_horizontal"), ("ML7", "SS_horizontal")),

        (("Control", "HS"), ("PTX", "HS")),
        (("Control", "SS_other"), ("PTX", "SS_other")),
        (("Control", "SS_horizontal"), ("PTX", "SS_horizontal")),
    ],
    test="t-test_welch",
    text_format="simple",
    loc="inside",
)

ax.set_ylabel('rel. length fluctuation')
# ax.set_ylim([0, 2])
ax.legend(loc='upper right')


# plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation_new.svg"))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(5, 8.5))

sns.violinplot(
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_running_stddev_norm',
    hue='class',
    hue_order=['HS', 'SS other', 'SS horizontal'],
    inner='box',
    bw=0.15
)

statannot.add_stat_annotation(
    ax,
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_running_stddev_norm',
    hue='class__',
    hue_order=['HS', 'SS_other', 'SS_horizontal'],
    box_pairs=[
        (("Control", "HS"), ("Control", "SS_horizontal")),
        (("Control", "HS"), ("Control", "SS_other")),

        (("ML7", "HS"), ("ML7", "SS_horizontal")),
        (("ML7", "HS"), ("ML7", "SS_other")),

        (("PTX", "HS"), ("PTX", "SS_horizontal")),
        (("PTX", "HS"), ("PTX", "SS_other")),

        (("Control", "HS"), ("ML7", "HS")),
        (("Control", "SS_other"), ("ML7", "SS_other")),
        (("Control", "SS_horizontal"), ("ML7", "SS_horizontal")),

        (("Control", "HS"), ("PTX", "HS")),
        (("Control", "SS_other"), ("PTX", "SS_other")),
        (("Control", "SS_horizontal"), ("PTX", "SS_horizontal")),
    ],
    test="t-test_welch",
    text_format="simple",
    loc="inside",
)

ax.set_ylabel('rel. length fluctuation')
ax.set_ylim([0, 2])
ax.legend(loc='upper right')


# plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation_new.svg"))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(5, 8.5))

sns.violinplot(
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_running_stddev_norm',
    hue='class',
    hue_order=['HS', 'SS other', 'SS horizontal'],
    inner='box',
    bw=0.15
)

statannot.add_stat_annotation(
    ax,
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_running_stddev_norm',
    hue='class__',
    hue_order=['HS', 'SS_other', 'SS_horizontal'],
    box_pairs=[
        (("Control", "HS"), ("Control", "SS_horizontal")),
        (("Control", "HS"), ("Control", "SS_other")),

        (("ML7", "HS"), ("ML7", "SS_horizontal")),
        (("ML7", "HS"), ("ML7", "SS_other")),

        (("PTX", "HS"), ("PTX", "SS_horizontal")),
        (("PTX", "HS"), ("PTX", "SS_other")),

        (("Control", "HS"), ("ML7", "HS")),
        (("Control", "SS_other"), ("ML7", "SS_other")),
        (("Control", "SS_horizontal"), ("ML7", "SS_horizontal")),

        (("Control", "HS"), ("PTX", "HS")),
        (("Control", "SS_other"), ("PTX", "SS_other")),
        (("Control", "SS_horizontal"), ("PTX", "SS_horizontal")),
    ],
    test="t-test_welch",
    text_format="simple",
    loc="inside",
)

ax.set_ylabel('rel. length fluctuation')
ax.set_ylim([0, 2])
ax.legend(loc='upper right')


# plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation_new.svg"))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=(5, 8.5))

sns.violinplot(
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_running_stddev_norm',
    hue='class',
    hue_order=['HS', 'SS other', 'SS horizontal'],
    inner='box',
    bw=0.15
)

statannot.add_stat_annotation(
    ax,
    data=tracked_bonds,
    x='Experiment',
    y='bond_length_running_stddev_norm',
    hue='class__',
    hue_order=['HS', 'SS_other', 'SS_horizontal'],
    box_pairs=[
        (("Control", "HS"), ("Control", "SS_horizontal")),
        (("Control", "HS"), ("Control", "SS_other")),

        (("ML7", "HS"), ("ML7", "SS_horizontal")),
        (("ML7", "HS"), ("ML7", "SS_other")),

        (("PTX", "HS"), ("PTX", "SS_horizontal")),
        (("PTX", "HS"), ("PTX", "SS_other")),

        (("Control", "HS"), ("ML7", "HS")),
        (("Control", "SS_other"), ("ML7", "SS_other")),
        (("Control", "SS_horizontal"), ("ML7", "SS_horizontal")),

        (("Control", "HS"), ("PTX", "HS")),
        (("Control", "SS_other"), ("PTX", "SS_other")),
        (("Control", "SS_horizontal"), ("PTX", "SS_horizontal")),
    ],
    test="t-test_welch",
    text_format="simple",
    loc="inside",
)

ax.set_ylabel('rel. length fluctuation')
ax.set_ylim([0, 2])
ax.legend(loc='upper right')


# plt.savefig("{}/{}".format(figure_path, "bond_length_fluctuation_w_orientation_new.svg"))

In [ ]:
data = tracked_bonds.loc[tracked_bonds['Experiment'] == 'Control']

sns.histplot(
    data.loc[data['type'] == 'SS'],
    x='bond_orientation_mean',
    y='bond_length_stddev_norm',
)

# Polarity

In [ ]:
Experiment = 'Control'
sample_id = 2
frame = cells.loc[cells['Experiment'] == Experiment]
frame = frame.loc[frame['sample_id'] == sample_id]

# overwrite cilia by available data from single frame
for track_id in np.unique(frame['track_id_cells']):
    tracked_cell = frame.loc[frame['track_id_cells'] == track_id]

    cilium = tracked_cell.loc[~np.isnan(tracked_cell['cilium_phi'])]
    if cilium.shape[0] == 0:
        continue
    if cilium.shape[0] > 1:
        raise RuntimeError("Many cilia found")

    cilium = cilium.iloc[0]

    frame.loc[frame['track_id_cells'] == track_id, 'cilium_phi'] = cilium['cilium_phi']
    frame.loc[frame['track_id_cells'] == track_id, 'cilium_phi_corrected'] = cilium['cilium_phi_corrected']
    frame.loc[frame['track_id_cells'] == track_id, 'cilium_rho'] = cilium['cilium_rho']
    frame.loc[frame['track_id_cells'] == track_id, 'cilium_rho_normalized'] = cilium['cilium_rho_normalized']
    frame.loc[frame['track_id_cells'] == track_id, 'cilium_rho_corrected'] = cilium['cilium_rho_corrected']
    frame.loc[frame['track_id_cells'] == track_id, 'cilium_DX'] = cilium['cilium_DX']
    frame.loc[frame['track_id_cells'] == track_id, 'cilium_DY'] = cilium['cilium_DY']

frame.loc[~np.isnan(frame['cilium_phi'])]

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=[1.4, 1.4])
plt.tight_layout()

# sample_id = 2
Experiment = 'Control'
HS_bonds = tracked_bonds.loc[tracked_bonds['type'] == 'HS']
# HS_bonds = HS_bonds.loc[HS_bonds['sample_id'] == sample_id]
HS_bonds = HS_bonds.loc[HS_bonds['Experiment'] == Experiment]

frame = cells.loc[cells['sample_id']==sample_id]
frame = frame.loc[frame['Experiment']==Experiment]
frame = frame.loc[~frame['is_border_cell']]
HS_bonds['is_close'] = HS_bonds['cilium_angular_distance__mean'] < math.pi / 2

sns.violinplot(
    data=HS_bonds,
    x='is_close', order=[True, False],
    y='bond_length_stddev_norm',
    inner='box',
    palette=['red', 'red'],
    hue='type',
    linewidth=linewidth,
    # bw=0.15
)

box_pairs=[
    (False, True)
#     (("Control", "HS"), ("Control", "SS")),
#     (("ML7", "SS"), ("ML7", "HS")),
#     (("Control", "SS"), ("ML7", "SS")),
#     (("Control", "HS"), ("ML7", "HS")),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=HS_bonds,
    x='is_close',
    y='bond_length_stddev_norm',
)
annotator.configure(
    test='t-test_welch',
    # test='Mann-Whitney',
    line_width=linewidth,
    # text_format='simple',
    # pvalue_format=[[1e-5, "1e-5"], [1e-4, "1e-4"], [1e-3, "0.001"], [1e-2, "0.01"], [5e-2, "0.05"]]
)
annotator.apply_and_annotate()


ax.set_ylabel('rel. length fluctuation')
ax.set_xlabel('Angle to cilium')
ax.set_ylim([0, 1.])
ax.set_xticklabels([r'$<\dfrac{\pi}{2}$',r'$\geq\dfrac{\pi}{2}$'])
ax.set_xticklabels([r'$<90\degree$',r'$\geq90\degree$'])
ax.set_yticks([0., 0.5, 1.])

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, ['HC-SC'], title='', loc='upper left')


plt.savefig("{}/{}".format(figure_path,
                           "bond_length_fluctuation_Control_HC_angle.svg"))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=[1.7, 1.7])
plt.tight_layout()

HS_bonds = tracked_bonds.loc[tracked_bonds['type'] == 'HS']
mask = np.where(
    HS_bonds['Experiment'] == 'Control',
    True,
    HS_bonds['Experiment'] == 'PTX',
)
data = HS_bonds.loc[mask]

frame = cells.loc[cells['sample_id']==sample_id]
frame = frame.loc[frame['Experiment']==Experiment]
frame = frame.loc[~frame['is_border_cell']]
data['is_close'] = data['cilium_angular_distance__mean'] < math.pi / 2

order=['Control', 'PTX']
sns.violinplot(
    data=data,
    x='Experiment', order=order,
    y='bond_length_stddev_norm',
    hue='is_close', hue_order=[True, False],
    inner='box',
    palette=['red', 'orange'],
    linewidth=linewidth,
    # bw=0.15
)

box_pairs=[
    (("Control", True), ("Control", False)),
    (("PTX", True), ("PTX", False)),
    (("Control", True), ("PTX", True)),
    (("Control", False), ("PTX", False)),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=data,
    x='Experiment', order=order,
    y='bond_length_stddev_norm',
    hue='is_close', hue_order=[True, False],
)
annotator.configure(
    test='Mann-Whitney',
    line_width=linewidth,
)
annotator.apply_and_annotate()


ax.set_ylabel('rel. length fluctuation')
ax.set_xlabel('')
ax.set_xticklabels(order, fontsize=labelsize)
ax.set_ylim([0, 1.])
# ax.set_xticklabels([r'$<\dfrac{\pi}{2}$',r'$\geq\dfrac{\pi}{2}$'])
ax.set_yticks([0., 0.5, 1.])


leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(
    leg_handles,
    [r'$<90\degree$', r'$\geq90\degree$'],
    fontsize=ticksize,
    title='',
    loc='upper left',
    bbox_to_anchor=(1,1.025)
)


plt.savefig("{}/{}".format(figure_path,
                           "bond_length_fluctuation_Control_HC_angle__PTX.svg"))

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=[1.7, 3.176])
plt.tight_layout()

HS_bonds['is_close_nematic'] = np.where(
    HS_bonds['cilium_angular_distance__mean'] < math.pi / 4,
    True,
    HS_bonds['cilium_angular_distance__mean'] > 3 * math.pi / 4
)

sns.violinplot(
    data=HS_bonds,
    y='bond_length_stddev_norm',
    x='is_close_nematic',
    inner='box',
    # palette=['red', 'gray'],
    # bw=0.15
)

box_pairs=[
    (False, True)
#     (("Control", "HS"), ("Control", "SS")),
#     (("ML7", "SS"), ("ML7", "HS")),
#     (("Control", "SS"), ("ML7", "SS")),
#     (("Control", "HS"), ("ML7", "HS")),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=HS_bonds,
    x='is_close_nematic',
    y='bond_length_stddev_norm',
)
annotator.configure(
    test='Mann-Whitney',
    line_width=0.75,
)
annotator.apply_and_annotate()


plt.savefig("{}/{}".format(figure_path,
                           "bond_length_fluctuation_Control_HC_angle__nematic.svg"))

# Polarity on SC-SC junction

In [ ]:
tracked_bonds.loc[tracked_bonds['type'] == 'SS']['cilium_angular_distance__mean']

In [ ]:
fix, ax = plt.subplots(1, 1, figsize=[1.7, 1.7])
plt.tight_layout()

SS_bonds = tracked_bonds.loc[tracked_bonds['type'] == 'SS']
mask = np.where(
    SS_bonds['Experiment'] == 'Control',
    True,
    SS_bonds['Experiment'] == 'PTX',
)
data = SS_bonds.loc[mask]

percentil = 30
# data.loc[~np.isnan(data['cilium_rho__mean']), 'is_close'] = data.loc[~np.isnan(data['cilium_rho__mean']), 'cilium_rho__mean'] < np.percentile(data.loc[~np.isnan(data['cilium_rho__mean']), 'cilium_rho__mean'], percentil)
data.loc[~np.isnan(data['cilium_angular_distance__mean']), 'is_close'] = data.loc[~np.isnan(data['cilium_angular_distance__mean']), 'cilium_angular_distance__mean'] < np.percentile(data.loc[~np.isnan(data['cilium_angular_distance__mean']), 'cilium_angular_distance__mean'], percentil)
# data['is_close'] = data['cilium_angular_distance__mean'] < math.pi / 2

print(data.loc[data['cilium_angular_distance__mean'] > math.pi / 2, ['Experiment', 'cilium_angular_distance__mean', 'is_close']])

sns.violinplot(
    data=data,
    x='Experiment', order=['Control', 'PTX'],
    y='bond_length_stddev_norm',
    hue='is_close', hue_order=[True, False],
    inner='box',
    palette=['red', 'orange'],
    linewidth=linewidth,
    # bw=0.15
)

box_pairs=[
    (("Control", True), ("Control", False)),
    (("PTX", True), ("PTX", False)),
    (("Control", True), ("PTX", True)),
    (("Control", False), ("PTX", False)),
]

annotator = Annotator(
    ax,
    box_pairs,
    data=data,
    x='Experiment', order=['Control', 'PTX'],
    y='bond_length_stddev_norm',
    hue='is_close', hue_order=[True, False],
)
annotator.configure(
    test='Mann-Whitney',
    line_width=linewidth,
)
annotator.apply_and_annotate()


ax.set_ylabel('rel. length fluctuation')
# ax.set_xlabel('Angle to cilium')
ax.set_ylim([0, 1.])
# ax.set_xticklabels([r'$<\dfrac{\pi}{2}$',r'$\geq\dfrac{\pi}{2}$'])
ax.set_yticks([0., 0.5, 1.])
# ax.legend(loc='upper right')

leg_handles = ax.get_legend_handles_labels()[0]
ax.legend(leg_handles, [r'$<90\degree$', r'$\geq90\degree$'], title='SC-SC circular distance', loc='upper left', bbox_to_anchor=(1,1.025))



# mpl.pyplot.legend(bbox_to_anchor=(1,1.025))

In [ ]:
sns.histplot(
    data=data,
    x='cilium_angular_distance__mean',
    hue='is_close',
    # binrange=[0, 30],
    multiple='stack',
)

In [ ]:
for frame in np.unique(bonds.loc[~np.isnan(bonds['cilium_angular_distance']), 'filename']):
    frame_bonds = bonds.loc[bonds['filename']==frame]
    fig, ax = plt.subplots(1, 1)
    # color = np.where(frame_bonds['type'] == 'SS', 'gray', 'black')
    # color = np.where(frame_bonds['type'] == 'HS', 'orange', color)
    # color = np.where(frame_bonds['type'] == 'HH', 'red', color)
    # color = np.where(frame_bonds['type'] == 'border', 'skyblue', color)
    # color = np.where(frame_bonds['type'] == 'border_plus_one',
    #                 'deepskyblue',
    #                 color)
    color=np.where(
        ~np.isnan(frame_bonds['cilium_angular_distance']),
        frame_bonds['cilium_angular_distance'],
        0
    )
    ax.quiver(
        frame_bonds['vx_1_x'],
        frame_bonds['vx_1_y'],
        frame_bonds['vx_2_x'] - frame_bonds['vx_1_x'],
        frame_bonds['vx_2_y'] - frame_bonds['vx_1_y'],
        color,
        cmap='seismic',
        clim=[0, math.pi],
        # vmin=0,
        # vmax=math.pi,
        scale_units='xy', angles='xy', scale=1,
        headwidth=0, headlength=0,headaxislength=0,
    )

    frame_cells = cells.loc[cells['filename']==frame]
    rho = (frame_cells['area_cells']/ math.pi)**0.5
    ax.quiver(
        frame_cells['center_x_cells'],
        frame_cells['center_y_cells'],
        rho * frame_cells['cilium_rho_corrected'] * np.cos(frame_cells['cilium_phi_corrected']),
        rho * frame_cells['cilium_rho_corrected'] * np.sin(frame_cells['cilium_phi_corrected']),
        color='black',
        scale_units='xy', angles='xy', scale=1,
    )

    ax.set_title(frame)

# Temporal development

In [ ]:
g = sns.FacetGrid(
    bonds,
    # col="",
    row="Experiment",
    hue='type',
    hue_order=['SS', 'HS'],
    margin_titles=True,
    sharey="col", sharex=True)


g.map_dataframe(
    sns.lineplot,
    x="time",
    y="bond_length_in_px",
    markers=True, ci='sd', err_style='band')

g.set_xlabels("Time [min]")
g.set_ylabels("Bond length [$\mu$m]")

g.add_legend()

plt.savefig("{}/{}".format(figure_path, "bonds_length_development.svg"))


In [ ]:
g = sns.FacetGrid(
    cells,
    # col="",
    row="Experiment",
    hue='is_HC',
    # hue_order=['SS', 'HS'],
    margin_titles=True,
    sharey="col", sharex=True)


g.map_dataframe(
    sns.lineplot,
    x="time",
    y="normalized_area_cells",
    # style='sample_id',
    markers=True, ci='sd', err_style='band')

g.set_xlabels("Time [min]")
g.set_ylabels("Cell area [$\mu$m$^2$]")

g.add_legend()

plt.savefig("{}/{}".format(figure_path, "cell_area_development.svg"))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True)

sns.lineplot(
    data=bonds.loc[bonds['Experiment'] == 'Control'],
    x='time',
    y='bond_length_in_px',
    hue='type',
    hue_order=['SS', 'HS'],
    ax=ax1
)

sns.lineplot(
    data=bonds.loc[bonds['Experiment'] == 'ML7'],
    x='time',
    y='bond_length_in_px',
    hue='type',
    hue_order=['SS', 'HS'],
    ax=ax2
)

## Summary

In [ ]:
cells['area_change_rate'] = np.nan
def difference(cell, *, cells):
      if (cell['frame'] == 0):
            return np.nan
      
      ref = cells.loc[cells['frame'] == (cell['frame'] - 1)]
      ref = ref.loc[ref['track_id_cells'] == cell['track_id_cells']]
      if ref.shape[0] == 0:
            ref = cells.loc[cells['frame'] == (cell['frame'] - 5)]

            ref = ref.loc[ref['track_id_cells'] == cell['track_id_cells']]
            if ref.shape[0] == 0:
                  return np.nan

      ref = ref.iloc[0]

      delta = (cell['area_cells'] - ref['area_cells']) / abs(cell['area_cells'])
      delta /= cell['time'] - ref['time']

      return np.abs(delta)
cells['area_change_rate'] = cells.apply(difference, axis=1, cells=cells)

In [ ]:
def difference(bond, *, bonds):
      if (bond['frame'] == 0):
            return np.nan
      
      ref = bonds.loc[bonds['frame_nb'] == (bond['frame_nb'] - 1)]
      ref = ref.loc[ref['track_id_bonds'] == bond['track_id_bonds']]
      if ref.shape[0] == 0:
            return np.nan

      ref = ref.iloc[0]

      delta = bond['bond_length_in_px'] - ref['bond_length_in_px']
      delta /= bond['time'] - ref['time']
      delta /= np.abs(bond['bond_length_in_px'])

      return np.abs(delta)
bonds['length_change_rate'] = bonds.apply(difference, axis=1, bonds=bonds)

In [ ]:
print("N Cells:")
print(cells.loc[cells['time'] == 0].groupby(by='Experiment')[['time', 'track_id_cells']].count())
print(cells.loc[cells['time'] == 120].groupby(by='Experiment')[['time', 'track_id_cells']].count())
print(cells.loc[cells['time'] == 180].groupby(by='Experiment')[['time', 'track_id_cells']].count())
print(cells.loc[cells['time'] == 240].groupby(by='Experiment')[['time', 'track_id_cells']].count())
print(cells.loc[cells['time'] == 300].groupby(by='Experiment')[['time', 'track_id_cells']].count())
print(cells.loc[cells['time'] == 400].groupby(by='Experiment')[['time', 'track_id_cells']].count())

print("\n\nN Bonds:")
print(bonds.loc[bonds['time'] == 0].groupby(by='Experiment')[['time', 'track_id_bonds']].count())
print(bonds.loc[bonds['time'] == 120].groupby(by='Experiment')[['time', 'track_id_bonds']].count())
print(bonds.loc[bonds['time'] == 180].groupby(by='Experiment')[['time', 'track_id_bonds']].count())
print(bonds.loc[bonds['time'] == 240].groupby(by='Experiment')[['time', 'track_id_bonds']].count())
print(bonds.loc[bonds['time'] == 300].groupby(by='Experiment')[['time', 'track_id_bonds']].count())
print(bonds.loc[bonds['time'] == 400].groupby(by='Experiment')[['time', 'track_id_bonds']].count())

In [ ]:
bulk_bonds = bonds[bonds['type'] != 'border_plus_one'].dropna(how='all')
normalization_length = bulk_bonds.groupby(
    by=['filename'])['bond_length_in_px'].mean()
bonds['normalized_bond_length'] = bonds.apply(
    lambda b, *, normalization:
        b['bond_length_in_px'] / normalization[b['filename']],
    normalization=normalization_length, axis=1)

In [ ]:
rel_length = bonds
rel_length['object_type'] = 'bond'
rel_length['data'] = 'norm_bond_length'
rel_length['data_hue'] = bonds['type']
rel_length['data_values'] = rel_length['normalized_bond_length']
rel_length = rel_length.dropna(subset=['data_values'])
rel_length = rel_length.loc[rel_length['type'] != 'border_plus_one']
rel_length = rel_length.loc[rel_length['type'] != 'HH']

rel_length_HH = rel_length.loc[rel_length['type'] == 'HS']
rel_length_SS = rel_length.loc[rel_length['type'] == 'SS']

rel_length_HH = rel_length_HH.loc[rel_length_HH.apply(lambda x: (x['time'] == 10 or x['time'] == 60 or x['time'] == 150), axis=1)]
rel_length_SS = rel_length_SS.loc[rel_length_SS.apply(lambda x: x['time'] == 10 or x['time'] == 60 or x['time'] == 150, axis=1)]

rel_area = cells
rel_area['object_type'] = 'cell'
rel_area['data'] = 'norm_cell_area'
rel_area['data_hue'] = cells.apply(lambda cell: 'H' if cell['is_HC'] else 'S', axis=1)
rel_area['data_values'] = rel_area['normalized_area_cells']
rel_area = rel_area.dropna(subset=['data_values'])

rel_area_H = rel_area.loc[rel_area['is_HC']]
rel_area_S = rel_area.loc[~rel_area['is_HC']]
rel_area_H = rel_area_H.loc[rel_area_H.apply(lambda x: x['time'] == 10 or x['time'] == 60 or x['time'] == 150, axis=1)]
rel_area_S = rel_area_S.loc[rel_area_S.apply(lambda x: x['time'] == 10 or x['time'] == 60 or x['time'] == 150, axis=1)]


data_H = pd.concat([rel_length_HH, rel_area_H], axis=0, ignore_index=True)
data_S = pd.concat([rel_length_SS, rel_area_S], axis=0, ignore_index=True)

def histplot(*args, **kwargs):
    return sns.histplot(*args, multiple="dodge", **kwargs)


g = sns.FacetGrid(data_H, col="data", row="Experiment",
                  hue='time', margin_titles=True)
g.map_dataframe(histplot, x="data_values")

# g.set_ylabels("Rel. change")
# g.set_xlabels("Time [{}]".format(time_step_units))
g.add_legend()
g.set_titles("Hair cells")
plt.savefig("{}/{}.svg".format(figure_path, "summary_stats_HC"))


g = sns.FacetGrid(data_S, col="data", row="Experiment",
                  hue='time', margin_titles=True)
g.map_dataframe(histplot, x="data_values")

# g.set_ylabels("Rel. change")
# g.set_xlabels("Time [{}]".format(time_step_units))
g.add_legend()
g.set_titles("Support cells")

plt.savefig("{}/{}.svg".format(figure_path, "summary_stats_SC"))

## Available sets

In [ ]:
print("Available datasets: ")
for set in cells.columns:
  print("  {}".format(set))

In [ ]:
print("Available datasets in bonds: ")
for set in bonds.columns:
  print("  {}".format(set))

In [ ]:
print(cells['HC_normalized_area'].mean())
print(HC['num_neighbors'].mean())
print(SC['num_neighbors'].mean())
print(cells['num_neighbors'].mean())
print(HC['num_hair_neighbors'].mean())
